In [0]:
import pandas as pd
import numpy as np
import pickle
from numpy import array
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [0]:
#Building vocabulary and converting each statement into vector

def create_unique(text_arr):
  dictt = {}
  count = -1
  x = np.array(text_arr)
  
  text = np.unique(x)
  #print(text)
  for item in text:
    data = (item.lower().strip().split(" "))
    for item1 in data:
      if item1 not in dictt:
        count = count +1
        dictt[item1] = count
  
  print(dictt)  
  return dictt


def vectorize(text_arr, dictt):
  vectors = []
  #print(text_arr)
  for sentence in text_arr:
    vector = [0] * len(dictt)
    words = (sentence.lower().strip().split(" "))
    for word in words:
      index = dictt[word]
      vector[index] += 1
    
    vectors.append(vector)
    #print(sentence)
  print(array(vectors))
  return vectors
  
  
  
  
# text = ["Hello, my name is XYZ, and your name is ABC. Nice to meet you.", "Lol, what is your name?"]
# dictt = create_unique(text)
# vector = vectorize(text, dictt)


In [0]:
class LogisticRegression:
  
  #initializing object
  def __init__(self, lr, num_iter, threshold = 0.5):
        self.lr = lr
        self.num_iter = num_iter
        self.threshold = threshold
  
  #adding X0 to X matrix, feature matrix
  def __add_intercept(self, X):
    ones = np.ones((X.shape[0],1))
    print(ones.shape)
    print(X.shape)
    return np.concatenate((ones,X), axis=1)
  
  #defining sigmoid funcion
  def _sigmoid(self,X):
    return (1/(1+np.exp(-X)))
  
  #defining loss function
  def __loss(self, h, y):
    first_term = -y*np.log(h)
    second_term = -(1-y)*np.log(1-h)
    return (first_term + second_term).mean()
  
  #training and deriving the theta vector
  def _train(self,X,y):
    X = self.__add_intercept(X)
    
    #Initializing theta vector as zeroes
    self.theta = np.zeros(X.shape[1])
    
    #making model equation and doing gradient descent
    for i in range(self.num_iter):
      z = np.dot(X,self.theta)
      h = self._sigmoid(z)
      
      gradient = np.dot(X.T,(h-y))/y.size
      self.theta -= self.lr*gradient
      
      z = np.dot(X,self.theta)
      h = self._sigmoid(z)
      loss = self.__loss(h,y)
      
      if i%50==0:
        print("iteration: ",i,"\t cost: ",loss)
      
    file = open("spam_model.pickle", "wb")
    pickle.dump(self.theta, file)
    file.close()
    
    return True
        
  def predict(self, X):
    
    file = open('spam_model.pickle','rb')
    self.theta = pickle.load(file)
    
    X = self.__add_intercept(X)
    pred = self._sigmoid((np.dot(X,self.theta)))
    
    return pred>=self.threshold
        

In [24]:
data = pd.read_csv('emails.csv')

dicct = create_unique(data.text.values.tolist())
x = np.array(vectorize(data.text.values.tolist(),dicct))

file = open("dictionary.pickle", "wb")
pickle.dump(dicct, file)
file.close()

# x = np.array((data.text.values))
y = np.array((data.spam.values))

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)


{'subject:': 0, '': 1, 'http': 2, ':': 3, '/': 4, 'www': 5, '.': 6, 'virtu': 7, 'ally': 8, '-': 9, 'anywhere': 10, 'com': 11, 'sports': 12, 'hello': 13, ',': 14, 'i': 15, 'was': 16, 'hoping': 17, 'you': 18, 'could': 19, 'help': 20, 'me': 21, 'the': 22, 'link': 23, 'above': 24, 'takes': 25, 'to': 26, 'several': 27, 'facility': 28, 'stadiumtours': 29, 'created': 30, 'by': 31, 'virtually': 32, 'interactive': 33, 'would': 34, 'like': 35, 'introduce': 36, 'concept': 37, 'of': 38, 'a': 39, 'virtual': 40, 'tour': 41, 'appropriate': 42, 'people': 43, 'at': 44, 'your': 45, 'organization': 46, '(': 47, 'our': 48, 'current': 49, 'customers': 50, "'": 51, 'premium': 52, 'seating': 53, 'and': 54, 'ticket': 55, 'sales': 56, 'marketing': 57, 'pr': 58, 'business': 59, 'development': 60, 'departments': 61, 'are': 62, 'having': 63, 'great': 64, 'success': 65, 'with': 66, 'their': 67, 'tours': 68, 'there': 69, 'may': 70, 'beinteresting': 71, 'sponsorship': 72, 'opportunities': 73, 'as': 74, 'well': 75, '

In [44]:
log = LogisticRegression(lr=0.01, num_iter=101)
log._train(x_train,y_train)

pr = log.predict(x_test)
accuracy_score(pr,y_test)

(4582, 1)
(4582, 37442)
iteration:  0 	 cost:  1.142278544422348
iteration:  50 	 cost:  0.30119856374860193
iteration:  100 	 cost:  0.2521418929351193
(1146, 1)
(1146, 37442)


0.9075043630017452